<a href="https://colab.research.google.com/github/grandending/AI-2023/blob/main/SAM_back%2Bfront_training_5_val_5_epoch100_%ED%95%99%EC%8A%B5%EC%A0%95%ED%99%95%EB%8F%84_90_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from traitlets.traitlets import isidentifier
import os
import pandas as pd
import cv2

mask_dir = '/content/drive/Shareddrives/AI/training_dataset/full_training_mask'

csv_path = '/content/drive/MyDrive/training/sorted_merged_label.csv'
df = pd.read_csv(csv_path, header=None)

X=[]
y_height = []
y_weight = []

mask_files = sorted([os.path.join(mask_dir, f) for f in os.listdir(mask_dir) if os.path.isfile(os.path.join(mask_dir, f))])
for mask_file in mask_files:
  file_name = os.path.basename(mask_file)
  identifier = file_name.split('_')[2]

  row = df[df[2].str.contains(identifier)]

  if row.empty:
    print(f"Warning: Identifier {identifier} not found in CSV.")
    continue

  try:
    height = float(row.iloc[0,3])
    weight = float(row.iloc[0,35])
  except ValueError:
    print(f"Invalid data for identifier {identifier}: height={row.iloc[0,3]}, weight={row.iloc[0,35]}")
    continue


  mask_path = os.path.join(mask_dir, mask_file)
  mask_image = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # 학습 데이터에 추가
  X.append(mask_image)
  y_height.append(height)
  y_weight.append(weight)

print(mask_files)
print(len(mask_files))


['/content/drive/Shareddrives/AI/training_dataset/full_training_mask/01_01_F009_04_segmented.jpg', '/content/drive/Shareddrives/AI/training_dataset/full_training_mask/01_01_F009_19_segmented.jpg', '/content/drive/Shareddrives/AI/training_dataset/full_training_mask/01_01_F010_04_segmented.jpg', '/content/drive/Shareddrives/AI/training_dataset/full_training_mask/01_01_F010_19_segmented.jpg', '/content/drive/Shareddrives/AI/training_dataset/full_training_mask/01_01_F011_04_segmented.jpg', '/content/drive/Shareddrives/AI/training_dataset/full_training_mask/01_01_F011_19_segmented.jpg', '/content/drive/Shareddrives/AI/training_dataset/full_training_mask/01_01_F012_04_segmented.jpg', '/content/drive/Shareddrives/AI/training_dataset/full_training_mask/01_01_F012_19_segmented.jpg', '/content/drive/Shareddrives/AI/training_dataset/full_training_mask/01_01_F013_04_segmented.jpg', '/content/drive/Shareddrives/AI/training_dataset/full_training_mask/01_01_F013_19_segmented.jpg', '/content/drive/Sha

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import cv2

class CustomDataset(Dataset):
    def __init__(self, img_paths, heights, weights, transform=None):
        self.img_paths = img_paths
        self.heights = heights
        self.weights = weights
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = os.path.join(mask_dir, self.img_paths[idx])
        img = cv2.imread(img_path)

        if img is None:
            raise FileNotFoundError(f"Unable to load image at path: {img_path}")

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if self.transform:
            img = self.transform(img)

        height = self.heights[idx]
        weight = self.weights[idx]

        return img, torch.tensor([height, weight], dtype=torch.float32)

In [ ]:
# 이미지 전처리를 위한 transform 정의
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),  # ResNet의 입력 이미지 크기에 맞게 조정
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# CustomDataset 인스턴스 생성
dataset = CustomDataset(mask_files, y_height, y_weight, transform=transform)

# DataLoader 인스턴스 생성
batch_size = 32
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
#resnet101

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 전이 학습을 위해 사전 훈련된 ResNet 모델 불러오기
model = models.resnet101(pretrained=True)

# 키 정보를 결합하기 위한 선형 레이어 추가 전에 num_ftrs 설정
num_ftrs = model.fc.in_features + 1

# 마지막 분류 층 제거
model = nn.Sequential(*list(model.children())[:-1])

fc = nn.Linear(num_ftrs, 1).to(device)  # 출력은 몸무게 하나이므로 1
# 모델을 GPU로 옮기기
model = model.to(device)

# 손실 함수 및 옵티마이저 설정
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


BACK +FRONT 같이 학습 EPOCH 70 +30

In [ ]:
import time

# 학습 루프
num_epochs = 30
model.train()



for epoch in range(num_epochs):

    start_time = time.time()

    running_loss = 0.0

    correct_predictions = 0
    total_samples = 0

    for imgs, labels in train_loader:
        imgs = imgs.to(device)
        heights = labels[:, 0].view(-1, 1).to(device)
        weights = labels[:, 1].to(device)

        features = model(imgs)
        features = features.view(features.size(0), -1)

        inputs = torch.cat((features, heights), dim=1)

        outputs = fc(inputs)
        loss = criterion(outputs.view(-1), weights)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * imgs.size(0)

        # 상대 오차가 5% 이내인 경우를 정확한 예측으로 간주
        relative_error = torch.abs((outputs.view(-1) - weights) / weights)
        correct_predictions += torch.sum(relative_error < 0.05).item()
        total_samples += weights.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_time = time.time() - start_time # 경과 시간 계산

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Time : {epoch_time:.4f} seconds")

    # 정확도 계산
    accuracy = correct_predictions / total_samples
    print(f"Accuracy: {accuracy * 100:.2f}%")

torch.save(model.state_dict(), 'model_weights.pth')

Epoch 1/30, Loss: 5.7050, Time : 151.8916 seconds
Accuracy: 81.01%
Epoch 2/30, Loss: 5.6917, Time : 152.1017 seconds
Accuracy: 81.90%
Epoch 3/30, Loss: 5.7804, Time : 151.4205 seconds
Accuracy: 81.23%
Epoch 4/30, Loss: 4.9079, Time : 151.8160 seconds
Accuracy: 86.16%
Epoch 5/30, Loss: 5.5995, Time : 149.5648 seconds
Accuracy: 82.18%
Epoch 6/30, Loss: 5.9323, Time : 151.4559 seconds
Accuracy: 81.79%
Epoch 7/30, Loss: 6.9971, Time : 151.1114 seconds
Accuracy: 75.46%
Epoch 8/30, Loss: 5.9884, Time : 151.3555 seconds
Accuracy: 81.23%
Epoch 9/30, Loss: 4.5657, Time : 151.1109 seconds
Accuracy: 86.61%
Epoch 10/30, Loss: 5.1794, Time : 150.2417 seconds
Accuracy: 83.47%
Epoch 11/30, Loss: 4.5802, Time : 152.6283 seconds
Accuracy: 87.51%
Epoch 12/30, Loss: 4.8109, Time : 152.2777 seconds
Accuracy: 84.76%
Epoch 13/30, Loss: 4.3607, Time : 153.4630 seconds
Accuracy: 88.35%
Epoch 14/30, Loss: 4.2700, Time : 153.7429 seconds
Accuracy: 88.91%
Epoch 15/30, Loss: 6.4103, Time : 152.2098 seconds
Accura

----------------------validation-----------------------------------


In [ ]:
from traitlets.traitlets import isidentifier
import os
import pandas as pd
import cv2

val_mask_dir = '/content/drive/Shareddrives/AI/validation_dataset/validation_mask'

val_csv_path = '/content/drive/MyDrive/validation/val_sorted_merged_label.csv'
df = pd.read_csv(val_csv_path, header=None)

val_mask_images=[]
val_height = []
val_weight = []

val_mask_files = sorted([os.path.join(val_mask_dir, f) for f in os.listdir(val_mask_dir) if os.path.isfile(os.path.join(val_mask_dir, f))])

for mask_file in val_mask_files:
  file_name = os.path.basename(mask_file)

  identifier = file_name.split('_')[2]


  row = df[df[2].str.contains(identifier)]

  if row.empty:
    print(f"Warning: Identifier {identifier} not found in CSV.")
    continue

  try:
    height = float(row.iloc[0,3])
    weight = float(row.iloc[0,35])
  except ValueError:
    print(f"Invalid data for identifier {identifier}: height={row.iloc[0,3]}, weight={row.iloc[0,35]}")
    continue


  mask_path = os.path.join(val_mask_dir, mask_file)
  mask_image = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # 학습 데이터에 추가
  val_mask_images.append(mask_image)
  val_height.append(height)
  val_weight.append(weight)




In [ ]:
# CustomDataset 인스턴스 생성
val_dataset = CustomDataset(val_mask_files, val_height, val_weight, transform=transform)

# Validation DataLoader
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# 모델을 평가 모드로 설정
model.eval()

running_loss = 0.0
correct_predictions = 0
total_samples = 0

# Validation Loop
with torch.no_grad():
    for imgs, labels in val_loader:
        imgs = imgs.to(device)
        heights = labels[:, 0].view(-1, 1).to(device)
        weights = labels[:, 1].to(device)

        features = model(imgs)
        features = features.view(features.size(0), -1)
        inputs = torch.cat((features, heights), dim=1)

        outputs = fc(inputs)
        loss = criterion(outputs.view(-1), weights)

        running_loss += loss.item() * imgs.size(0)


        relative_error = torch.abs((outputs.view(-1) - weights) / weights)
        correct_predictions += torch.sum(relative_error < 0.05).item()
        total_samples += weights.size(0)

val_loss = running_loss / len(val_loader.dataset)
print(f"Validation Loss: {val_loss:.4f}")

# 정확도 계산
accuracy = correct_predictions / total_samples
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

Validation Loss: 28.0753
Validation Accuracy: 62.50%


In [ ]:
model.eval()

running_loss = 0.0
correct_predictions = 0
total_samples = 0

# Validation Loop
with torch.no_grad():
    for imgs, labels in val_loader:
        imgs = imgs.to(device)
        heights = labels[:, 0].view(-1, 1).to(device)
        weights = labels[:, 1].to(device)

        features = model(imgs)
        features = features.view(features.size(0), -1)
        inputs = torch.cat((features, heights), dim=1)

        outputs = fc(inputs)
        loss = criterion(outputs.view(-1), weights)

        running_loss += loss.item() * imgs.size(0)

        relative_error = torch.abs((outputs.view(-1) - weights) / weights)
        correct_predictions += torch.sum(relative_error < 0.05).item()
        total_samples += weights.size(0)

        # 각 이미지에 대한 예측 결과 출력
        for i in range(len(outputs)):
            print(f"Image {i+1}: True Weight = {weights[i].item()}, Predicted Weight = {outputs[i].item()}")

val_loss = running_loss / len(val_loader.dataset)
print(f"Validation Loss: {val_loss:.4f}")

# 정확도 계산
accuracy = correct_predictions / total_samples
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

Image 1: True Weight = 49.400001525878906, Predicted Weight = 48.03708267211914
Image 2: True Weight = 76.0999984741211, Predicted Weight = 64.60285186767578
Image 3: True Weight = 77.5999984741211, Predicted Weight = 78.7166519165039
Image 4: True Weight = 79.30000305175781, Predicted Weight = 72.96304321289062
Image 5: True Weight = 56.099998474121094, Predicted Weight = 57.74155044555664
Image 6: True Weight = 73.4000015258789, Predicted Weight = 72.8613510131836
Image 7: True Weight = 62.900001525878906, Predicted Weight = 69.68877410888672
Image 8: True Weight = 53.20000076293945, Predicted Weight = 53.48979949951172
Validation Loss: 28.0753
Validation Accuracy: 62.50%
